# Introduction to Python for Machine Learning

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statistics as st
import matplotlib.pyplot as plt
import re
import os

## 2. Data Importation/Download

#### 2.1. Create Directories and URL

In [ ]:
# Create url
url = 'http://github.com/WalePhenomenon/climate_change/blob/master/fuel_ferc1.csv?raw=true'

# variable to store file path
file_path = r'DataSets'

# variable to store file name
file_name = 'fuel_dataset.xlsx'

# variable to store file directory
file_directory = file_path+"\\"+file_name

# string variable to store sheet name
excel_sheet = 'DataSet'

# Create file directory
os.makedirs(os.path.dirname(file_directory), exist_ok=True)

#### 2.2. Import DataSet to Local directory

In [ ]:
try: # try load from file
    raw_df = pd.read_excel(file_directory, excel_sheet)
    print("Fuel DataSet Loaded!")
except FileNotFoundError: 
    try: # try download data set from git repo
        raw_df = pd.read_csv(url)
        # Save to local storage
        raw_df.to_excel(file_directory, excel_sheet, index=False) 
        print("File downloaded!") 
    except OSError: 
        print('Network Connection Error!')

## 3. Data Exploration 
###### Here we would try to understand our dataset

#### 3.1. Display the data set

In [ ]:
# Display first 4 rows
raw_df.head(4)

#### 3.2. Display dataset info

In [ ]:
# Get data set info
raw_df.info(verbose=True, null_counts=False)

#### 3.3. Describe the data set to understand the relations, dependencies or unique features

In [ ]:
# describe dataset
raw_df.describe(include = 'all')

#### 3.4. Check Are there null values (how many)?

In [ ]:
# Check for null values in dataset
raw_df.isna().sum()

#### 3.5. What correlations do this data set have?
###### -> This would enable us to understand how closely related some features are.

In [ ]:
# Check data correlation
raw_df.corr()

In [ ]:
"""-> The most correlation was between the 'report_year' and the 'utility_id_ferc1', while the least correlation was between the 'fuel_mmbtu_per_unit' and the 'report_year'.
There is a huge disjoint here, why?"""

#### 3.5. What are the least and most correlated features in the data set?

In [ ]:
# Check if there is any correlation between axis='fuel_cost_per_unit_delivered' 
# and axis='fuel_cost_per_unit_burned'
raw_df.corrwith(raw_df['fuel_cost_per_unit_delivered'])

#### 3.7. Probability distribution density of the dataset (i.e using kurtosis() and skewness())

In [ ]:
""" -> This would help in determining presence of outliers and column involved """

In [ ]:
# Check for the peak and tails of the dataset using pd.DataFrame().kurtosis()
raw_df.kurtosis(skipna=True,numeric_only=True) 

In [ ]:
# Check for the peak and tails of the dataset using pd.DataFrame().sew()
# to determine the outliers in the dataset
raw_df.skew() 

## 4. DataSet Cleaning and DataSet Wrangling

#### 4.1. Task 1: Clean 'plant_name_ferc1'

In [ ]:
# function to remove characters such as #,*,(,),&,. from the strings
def replace_char(line): 
    return line.translate({ord(c): '' for c in '.#*$&-()_  '}) 

In [ ]:
# apply function to axis='plant_name_ferc1' to remove characters
raw_df['plant_name_ferc1'] = raw_df['plant_name_ferc1'].apply(lambda x: replace_char(x))

# display first 50 rows in the series 'plant_name_ferc1'
raw_df['plant_name_ferc1'].head(5) 

In [ ]:
# Save modified dataframe to file
raw_df.to_excel(file_path+"\\"+'Modified.xlsx','Modified')

#### Task 2: remove null values in axis='fuel_unit'

In [ ]:
#display first 5 rows of dataframe
raw_df.head(5)

In [ ]:
# display null values contained in the data set
raw_df.isnull().sum()

In [ ]:
raw_df.groupby('fuel_type_code_pudl')['fuel_unit'].value_counts()

In [ ]:
# copy data frame
copy_df = raw_df
mode = []
fuel_class = pd.DataFrame()

fuel_type_list = copy_df['fuel_type_code_pudl'].unique()

for fuel_type in fuel_type_list:
    mode.append(copy_df.loc[copy_df['fuel_type_code_pudl']==fuel_type,'fuel_unit'].mode())

fuel_class['Fuel Types'] = fuel_type_list
fuel_class['Fuel Units'] = pd.DataFrame(mode)
fuel_class

In [ ]:
 copy_df['fuel_type_code_pudl']   

In [ ]:
# remove null values from axis='fuel_unit' by grouping axis='fuel_type_code_pudl'
for index,fuel_type in fuel_class.iterrows():
    print(fuel_type['Fuel Units'])

In [ ]:
copy_df.isna().any()

#### P

In [ ]:
# remove null values from axis='fuel_unit' by grouping axis='fuel_type_code_pudl'
for index,fuel_type in fuel_class.iterrows():
    copy_df.loc[copy_df['fuel_type_code_pudl']==fuel_type['Fuel Types'],'fuel_unit'] = copy_df.loc[copy_df['fuel_type_code_pudl']==fuel_type['Fuel Types'],'fuel_unit'].fillna(value=fuel_type['Fuel Units'])

copy_df.isnull().sum()

In [ ]:
# Save modified dataframe to file
copy_df.to_excel(file_path+"\\"+'Modified_1.xlsx','Modified')

In [ ]:
copy_df.isna().any()

#### Task 3: Fill the rows containing '0' in axis='fuel_cost_per_unit_delivered'

##### convert 0 to nan

In [ ]:
copy_df.loc[copy_df['fuel_cost_per_unit_delivered'] == 0].head(5)

In [ ]:
copy_df1 = copy_df

In [ ]:
copy_df1.loc[copy_df1['fuel_cost_per_unit_delivered']== 0].head()

In [ ]:
# convert the '0' digits to null values
copy_df1.loc[copy_df1['fuel_cost_per_unit_delivered']== 0,'fuel_cost_per_unit_delivered'] = np.nan

# Check if conversion was successful
copy_df1.isnull().sum()

In [ ]:
# display the first 50 rows
copy_df1.head(20)

In [ ]:
# Save modified dataframe to file
copy_df1.to_excel(file_path+"\\"+'Modified_2.xlsx','Modified')

In [ ]:
# 
copy_df1['fuel_cost_per_unit_delivered'].interpolate(method='values',inplace=True)

In [ ]:
copy_df1.isnull().sum()

In [ ]:
copy_df1.head(20)

In [ ]:
# Save modified dataframe to file
copy_df1.to_excel(file_path+"\\"+'Modified_3.xlsx','Modified')

## Data Visualization

#### What is the cost of fuel burnt and delivered?

#### Which plant is the most efficient?

#### Which plant is the worst performing plant?

#### Which utilities are the best and worst performing

#### Which of the fuels are the most economical and efficient

#### What year was the best performing year independent of the cost of fuel per mmbtu?

#### What factors where responsible for the outstanding performance in question --- above?

#### Seeing the above what are your recommendations?